In [1]:
# import of necessary libaries
# MIKE IO 1D, needs Pandas and Numpy
import mikeio1d
from mikeio1d.res1d import Res1D, QueryDataNode, QueryDataReach,ResultData, mike1d_quantities, ResultData
import pandas as pd
import numpy as np

# connection to MIKE+ database:
import sqlite3

# file and folder manipulation for input and output:
import os

# find files faster
from fnmatch import fnmatch

In [2]:
# location of mikeio

# print(mikeio1d.__file__)

# Define functions

In [3]:
print('Defining functions...')

Defining functions...


In [4]:
# get Discharge of full reach:

def get_qfull_data(res1d):
    network_datas = list(res1d.data.NetworkDatas)
    for data in network_datas:
        if data.Quantity.Id == "Discharge of full reach":
            return data    

def get_qfull(reach, qfull_data):
    if qfull_data is None:
        return None

    reach_data = qfull_data.GetReachData(reach.Name)

    if reach_data is not None:
        return reach_data.GlobalValue

    return None

In [5]:
# return InvertLevel and GroundLevel from a Res1DManhole object:

def get_node_levels(node):
    try:
        invert_level = node.BottomLevel
    except:
        invert_level = None

    try:
        ground_level = node.GroundLevel
    except:
        ground_level = None

    return invert_level, ground_level

In [6]:
# return Diameter of reach looking at the first GridPoint, with two decimal places
# needs modification for rectangular and CRS reaches:

def get_diameter(reach):
    try:
        grid_points = list(reach.GridPoints)
        h_point = grid_points[0]
        diameter = round(h_point.CrossSection.Diameter,2)
    except:
        diameter = None

    return diameter

In [7]:
# compute slope from first and last GridPoint, returns absolute value in percent:

def get_slope(reach):
    grid_points = list(reach.GridPoints)
    gp_first = grid_points[0]
    gp_last = grid_points[-1]
    length = reach.Length
    slope = ((gp_first.Z - gp_last.Z) / length)*100
    return abs(slope)

In [8]:
# returns the time series (?) for a specific Quantity ID

def get_data_item(reach, quantity_id):
    item = None

    for data_item in list(reach.DataItems):
        if data_item.Quantity.Id == quantity_id:
            item = data_item
            break

    return item

In [9]:
# Get min and max value and times for a any model element (not only reaches)
# reach...DHI.Mike1D.ResultDataAccess.Res1DManhole object or similar
# timeslist...list of DateTime objects
# quantity_id...string with quantity (default ist "Discharge")

def get_minmax_value_result_file(reach, times_list, quantity_id="Discharge"):
    item = get_data_item(reach, quantity_id)

    min_value, min_time = None, None
    max_value, max_time = None, None   
    
    try:
        time_data = item.TimeData
        for time_step_index in range(time_data.NumberOfTimeSteps):
            
            for element_index in range(time_data.NumberOfElements):
                value = time_data.GetValue(time_step_index, element_index)
                if min_value is None or value < min_value:
                    min_value = value 
                    min_time = times_list[time_step_index].ToString()

                if max_value is None or value > max_value:
                    max_value = value 
                    max_time =  times_list[time_step_index].ToString()
    
    except:
        time_data = None
    
    return min_value, min_time, max_value, max_time

In [10]:
# not used; probably an early version of get_minmax_value_result_file

# def get_minmax_value(reach, simulation_start, quantity_id="Discharge"):    
#     item_max = get_data_item(reach, quantity_id + "Max")
#     item_max_time = get_data_item(reach, quantity_id + "MaxTime")
#     item_min = get_data_item(reach, quantity_id + "Min")
#     item_min_time = get_data_item(reach, quantity_id + "MinTime")

#     items = [item_max, item_max_time, item_min, item_min_time]
#     if None in items:
#         return None

#     min_value, min_time = None, None
#     max_value, max_time = None, None   

#     number_of_elements = item_min.TimeData.NumberOfElements
#     time_step_index = 0

#     for element_index in range(number_of_elements):
#         value = item_min.TimeData.GetValue(time_step_index, element_index)
#         time = item_min_time.TimeData.GetValue(time_step_index, element_index)
        
#         if min_value is None or value < min_value:
#             min_value = value 
#             min_time = simulation_start.AddSeconds(time)

#         value = item_max.TimeData.GetValue(time_step_index, element_index)
#         time = item_max_time.TimeData.GetValue(time_step_index, element_index)

#         if max_value is None or value > max_value:
#             max_value = value 
#             max_time = simulation_start.AddSeconds(time)

#     return min_value, min_time.ToString(), max_value, max_time.ToString()

In [11]:
# return max value and time for start and end of reach
# works probably not only for WaterLevel
# get_reach_start_values must be some predefined method for res1d

def max_WL_start_end (quantity):
    try:
        #np-array of waterlevels
        wl_start = res1d.get_reach_start_values(reach.Name, quantity)
        wl_end = res1d.get_reach_end_values(reach.Name, quantity)

        #max values
        wl_start_max = wl_start.max()
        wl_end_max = wl_end.max()

        #corresponding time
        time_start=str(res1d.time_index[np.argmax(wl_start)])
        time_end=str(res1d.time_index[np.argmax(wl_start)])
        
    except:
        wl_start,wl_end,wl_start_max,wl_end_max,time_start,time_end= None, None, None, None, None, None
    
    return wl_start_max, time_start, wl_end_max, time_end

In [12]:
# compute a/b but return None if a or b is None:

def get_ratio(a, b):
    try:
        ratio = a/b
        
    except:
        a is None or b is None
        ratio=None
    
    return ratio

In [13]:
# returns type of reach
# needs additional types like valve or orifice

def get_reach_type(reach):
    try:
        # get the full identifier string:
        fullstring = str(reach)
        
        # might be:
        # 'Res1DReach: B4.1520l1-13 (0-235.000625581015)'
        # 'Res1DReach: Weir:B4.1480w1-14 (0-1)',
        # 'Res1DReach: Pump:B4.1510p1-16 (0-80.0006092712283)'

        # get the second part after the ':'
        structureReach = fullstring.split(':')[1].lstrip()

        # if the second part is Pump or Weir, use Pump or Weir:
        if structureReach in ['Pump','Weir']:
            reach_type = structureReach

        # in any other case this must be a normal Link (Pipe or Canal or River)
        else:
            reach_type = 'Link'
    
    except:
        reach_type = None

    return reach_type

# Find res1d and sqlite files

In [14]:
print('Searching res1d and sqlite files...')

Searching res1d and sqlite files...


In [15]:
cwd = os.getcwd()

# cwd

In [16]:
myFiles = os.listdir(cwd)

In [17]:
# create a list of res1d-files
myRes1dFiles = [file for file in myFiles if fnmatch(file, '*.res1d')]

# pick the first res1d-file
oneRes1dFile = myRes1dFiles[0]

print('Current res1d-file: ' + oneRes1dFile)

Current res1d-file: Tutor1base.Copy.res1d


In [18]:
# create a list of sqlite-files
mySQLiteFiles = [file for file in myFiles if fnmatch(file, '*.sqlite')] 

# pick the first sqlite-file
oneSQLiteFile = mySQLiteFiles[0]

print('Current MIKE+ database: ' + oneSQLiteFile)

Current MIKE+ database: mp21Tutor1.sqlite


# Create nodes and reaches lists from res1d

In [19]:
print('Creating nodes and links lists from res1d...')

Creating nodes and links lists from res1d...


In [20]:
# create a Res1d-object
res1d = Res1D(oneRes1dFile)

reaches = list(res1d.data.Reaches)
nodes = list(res1d.data.Nodes)

qfull_data = get_qfull_data(res1d)
times_list  = list(res1d.data.TimesList)

simulation_start = res1d.data.StartTime

In [21]:
res1d

<mikeio1d.Res1D>
Start time: 1953-05-16 18:00:00
End time: 1953-05-16 21:59:05
# Timesteps: 157
# Catchments: 0
# Nodes: 16
# Reaches: 17
# Globals: 2
0 - WaterLevel <m>
1 - Discharge <m^3/s>
2 - TimeStep <sec>
3 - NumTriesForTimeStep <m>

In [22]:
reaches

In [23]:
nodes

# Prepare desired node results

In [24]:
print('Preparing desired node results...')

Preparing desired node results...


In [25]:
# initialize node lists

Node_ID = []
WLmin = []
WLmax = []
WLmaxmin = []

In [26]:
# call necessary informations from initially defined functions

for node in nodes:
    id = node.ID
    min_wl = get_minmax_value_result_file(node, times_list, quantity_id="WaterLevel")[0]
    max_wl = get_minmax_value_result_file(node, times_list, quantity_id="WaterLevel")[2]
    max_min_wl = max_wl / min_wl
    
    # fill initialized node lists    
    Node_ID.append(id)
    WLmin.append(min_wl)
    WLmax.append(max_wl)
    WLmaxmin.append(max_min_wl)
    

In [27]:
# create dictionary with node lists

dict_res1d_nodes = {'Node_ID':Node_ID, 'WLmin':WLmin, 'WLmax':WLmax, 'WLmaxmin':WLmaxmin}

In [28]:
# create dataframe from dictionaty

df_res1dNode = pd.DataFrame(dict_res1d_nodes)

df_res1dNode = df_res1dNode.set_index('Node_ID')

df_res1dNode

,WLmin,WLmax,WLmaxmin
Node_ID,,,
B4.1520,16.50250,19.50122,1.181713
B4.1500,16.70300,19.61063,1.174078
B4.1501,17.60150,20.76025,1.179459
B4.1502,17.86125,21.78367,1.219605
B4.1490,16.39500,17.43768,1.063597
B4.1485,16.35500,17.37703,1.062490
B4.1320,17.10500,17.51902,1.024205
B4.1310,17.30500,17.66968,1.021074
B4.1300,17.42500,17.88718,1.026524


# Prepare desired Link results

In [29]:
print('Preparing desired link results...')

Preparing desired link results...


In [30]:
#initialize lists
Link_ID=[]
Reach_Type=[]
From_Node_ID=[]
Invert_level_from_Node=[]
Ground_level_from_Node=[]
To_Node_ID=[]
Invert_level_to_Node=[]
Ground_level_to_Node=[]
Length=[]
Diameter=[]
Slope=[]
Qfull=[]

Vmin=[]
Vmin_time=[]
Vmax=[]
Vmax_time=[]
Qmin=[]
Qmin_time=[]
Qmax=[]
Qmax_time=[]

Qmax_Qfull=[]
WLmax_start=[]
WLmax_start_time=[]
WLmax_end=[]
WLmax_end_time=[]

In [31]:
# call necessary informations from initially defined functions

for reach in reaches:

    name = reach.Name
    reachtype = get_reach_type(reach)
    node_from_index = reach.StartNodeIndex
    node_to_index = reach.EndNodeIndex

    node_from = nodes[node_from_index]
    node_to = nodes[node_to_index]

    invert_level_from, ground_level_from = get_node_levels(node_from)
    invert_level_to, ground_level_to = get_node_levels(node_to)

    diameter = get_diameter(reach)

    slope = get_slope(reach)

    qfull = get_qfull(reach, qfull_data)

    v_minmax_data = get_minmax_value_result_file(reach, times_list, "FlowVelocity")
    q_minmax_data = get_minmax_value_result_file(reach, times_list, "Discharge")
       
    maxWL=max_WL_start_end('WaterLevel')

    Qmax_Qfull_ratio = get_ratio(q_minmax_data[2],qfull)

    #fill initialized lists
    Link_ID.append(name)
    Reach_Type.append(reachtype)
    From_Node_ID.append(node_from.ID)
    Invert_level_from_Node.append(invert_level_from)
    Ground_level_from_Node.append(ground_level_from)
    To_Node_ID.append(node_to.ID)
    Invert_level_to_Node.append(invert_level_to)
    Ground_level_to_Node.append(ground_level_to)
    Diameter.append(diameter)
    Length.append(reach.Length)
    Slope.append(slope)
    Vmin.append(v_minmax_data[0])
    Vmin_time.append(v_minmax_data[1])
    Vmax.append(v_minmax_data[2])
    Vmax_time.append(v_minmax_data[3])
    Qmin.append(q_minmax_data[0])
    Qmin_time.append(q_minmax_data[1])
    Qmax.append(q_minmax_data[2])
    Qmax_time.append(q_minmax_data[3])
    Qfull.append(qfull)
    Qmax_Qfull.append(Qmax_Qfull_ratio)
    WLmax_start.append(maxWL[0])
    WLmax_start_time.append(maxWL[1])
    WLmax_end.append(maxWL[2])
    WLmax_end_time.append(maxWL[3])
    

In [32]:
# create dictionary with link lists

dict_res1d_links = {'Link_ID':Link_ID, 'Reach_Type':Reach_Type, 'From_Node_ID':From_Node_ID,'Invert_level_from_Node':Invert_level_from_Node, 
                   'Ground_level_from_Node':Ground_level_from_Node,'To_Node_ID':To_Node_ID, 'Invert_level_to_Node': Invert_level_to_Node,
                   'Ground_level_to_Node':Ground_level_to_Node, 'Diameter': Diameter,'Length':Length, 'Slope':Slope,'Vmin':Vmin, 
                   'Vmin_time':Vmin_time,'Vmax':Vmax, 'Vmax_time':Vmax_time, 'Qmin':Qmin, 'Qmin_time':Qmin_time,'Qmax':Qmax, 'Qmax_time':Qmax_time,
                   'Qfull':Qfull,'Qmax_Qfull':Qmax_Qfull, 'WLmax_start':WLmax_start, 'WLmax_start_time':WLmax_start_time, 'WLmax_end':WLmax_end,
                   'WLmax_end_time':WLmax_end_time}

In [33]:
# create dataframe from dictionary

df_res1dLink=pd.DataFrame(dict_res1d_links)

# keep only those lines where ReacH_Type = 'Link'
# df_res1dLink = df_res1dLink.loc[df['Reach_Type'] == 'Link']
# currently obsolete, as below merge eliminates non-Links

# set index on Link_ID:
df_res1dLink =  df_res1dLink.set_index('Link_ID')

df_res1dLink.head()

,Reach_Type,From_Node_ID,Invert_level_from_Node,Ground_level_from_Node,To_Node_ID,Invert_level_to_Node,Ground_level_to_Node,Diameter,Length,Slope,...,Qmin,Qmin_time,Qmax,Qmax_time,Qfull,Qmax_Qfull,WLmax_start,WLmax_start_time,WLmax_end,WLmax_end_time
Link_ID,,,,,,,,,,,,,,,,,,,,,
B4.1200l1,Link,B4.1200,16.799999,19.900000,B4.1485,16.35,19.799999,0.7,479.998999,0.093750,...,-0.005379,16.05.1953 18:20:30,0.423308,16.05.1953 18:21:30,0.141865,2.983871,20.247484,1953-05-16 18:32:23,17.377028,1953-05-16 18:32:23
B4.1300l1,Link,B4.1300,17.420000,19.900000,B4.1320,17.10,20.200001,NaN,299.999956,0.106667,...,0.000191,16.05.1953 21:57:16,0.386263,16.05.1953 18:26:37,-1.000000,-0.386263,17.843431,1953-05-16 18:26:37,17.519020,1953-05-16 18:26:37
B4.1310l1,Link,B4.1310,17.299999,20.230000,B4.1320,17.10,20.200001,NaN,323.110187,0.061898,...,0.000146,16.05.1953 21:57:16,0.213843,16.05.1953 18:26:37,-1.000000,-0.213843,17.646738,1953-05-16 18:26:37,17.519020,1953-05-16 18:26:37
B4.1320l1,Link,B4.1320,17.100000,20.200001,B4.1485,16.35,19.799999,NaN,270.002056,0.277776,...,-0.228450,16.05.1953 18:25:30,0.430228,16.05.1953 18:30:35,-1.000000,-0.430228,17.444399,1953-05-16 18:32:23,17.377028,1953-05-16 18:32:23
B4.1480l1,Link,B4.1480,16.250000,18.500000,A0.0327,16.00,17.500000,1.2,119.998586,0.208336,...,-0.804957,16.05.1953 18:50:37,1.260161,16.05.1953 18:34:11,1.966383,0.640852,17.127731,1953-05-16 18:50:37,17.187971,1953-05-16 18:50:37


# Join Node results with msm_Node

In [34]:
print('Joining results with msm_Node information...')

Joining results with msm_Node information...


In [35]:
# establish connection to MIKE+ database:
con = sqlite3.connect(oneSQLiteFile)

In [36]:
# pick two columns from 'msm_Link':
df_msmNode = pd.read_sql_query("SELECT muid, description, assetname from msm_Node", con)

In [37]:
# set index on 'muid'
df_msmNode = df_msmNode.set_index('muid')

In [38]:
df_msmNode.head()

,description,assetname
muid,,
B4.1520,None,None
B4.1500,None,Nordknoten
B4.1501,None,None
B4.1502,None,Nordwestknoten
B4.1490,None,None


In [39]:
# join...werden dadurch die fälschlichen Knoten der Wehre entfernt?
df_res1dmsmNode = pd.merge(df_res1dNode, df_msmNode, left_index=True, right_index=True)

df_res1dmsmNode

,WLmin,WLmax,WLmaxmin,description,assetname
B4.1520,16.50250,19.50122,1.181713,None,None
B4.1500,16.70300,19.61063,1.174078,None,Nordknoten
B4.1501,17.60150,20.76025,1.179459,None,None
B4.1502,17.86125,21.78367,1.219605,None,Nordwestknoten
B4.1490,16.39500,17.43768,1.063597,None,None
B4.1485,16.35500,17.37703,1.062490,None,Kreuzknoten
B4.1320,17.10500,17.51902,1.024205,None,None
B4.1310,17.30500,17.66968,1.021074,None,Ostknoten
B4.1300,17.42500,17.88718,1.026524,None,None
B4.1200,16.80350,20.37670,1.212646,None,None


# Join Link results with msmLink

In [40]:
print('Joining Link results with msm_Link information...')

Joining Link results with msm_Link information...


In [41]:
# establish connection to MIKE+ database:
con = sqlite3.connect(oneSQLiteFile)

In [42]:
# pick two columns from 'msm_Link':
df_msmLink = pd.read_sql_query("SELECT muid, description, assetname from msm_Link", con)

In [43]:
# set index on 'muid'
df_msmLink = df_msmLink.set_index('muid')

In [44]:
df_msmLink.head()

,description,assetname
muid,,
B4.1200l1,None,None
B4.1300l1,None,None
B4.1310l1,None,None
B4.1320l1,None,None
B4.1480l1,None,None


In [45]:
df_res1dmsmLink = pd.merge(df_res1dLink, df_msmLink, left_index=True, right_index=True)

df_res1dmsmLink.head()

,Reach_Type,From_Node_ID,Invert_level_from_Node,Ground_level_from_Node,To_Node_ID,Invert_level_to_Node,Ground_level_to_Node,Diameter,Length,Slope,...,Qmax,Qmax_time,Qfull,Qmax_Qfull,WLmax_start,WLmax_start_time,WLmax_end,WLmax_end_time,description,assetname
B4.1200l1,Link,B4.1200,16.799999,19.900000,B4.1485,16.35,19.799999,0.7,479.998999,0.093750,...,0.423308,16.05.1953 18:21:30,0.141865,2.983871,20.247484,1953-05-16 18:32:23,17.377028,1953-05-16 18:32:23,None,None
B4.1300l1,Link,B4.1300,17.420000,19.900000,B4.1320,17.10,20.200001,NaN,299.999956,0.106667,...,0.386263,16.05.1953 18:26:37,-1.000000,-0.386263,17.843431,1953-05-16 18:26:37,17.519020,1953-05-16 18:26:37,None,None
B4.1310l1,Link,B4.1310,17.299999,20.230000,B4.1320,17.10,20.200001,NaN,323.110187,0.061898,...,0.213843,16.05.1953 18:26:37,-1.000000,-0.213843,17.646738,1953-05-16 18:26:37,17.519020,1953-05-16 18:26:37,None,None
B4.1320l1,Link,B4.1320,17.100000,20.200001,B4.1485,16.35,19.799999,NaN,270.002056,0.277776,...,0.430228,16.05.1953 18:30:35,-1.000000,-0.430228,17.444399,1953-05-16 18:32:23,17.377028,1953-05-16 18:32:23,None,None
B4.1480l1,Link,B4.1480,16.250000,18.500000,A0.0327,16.00,17.500000,1.2,119.998586,0.208336,...,1.260161,16.05.1953 18:34:11,1.966383,0.640852,17.127731,1953-05-16 18:50:37,17.187971,1953-05-16 18:50:37,None,None


# define output res1d-report format (*.csv | *.xlsx)

In [46]:
print('Preparing your output...')

Preparing your output...


In [47]:
# extract the root of the filename even if filename contains a dot, hence better than split('.')
rootName = os.path.splitext(oneRes1dFile)[0]

In [54]:
# export node results

# MUID is index and always exported as the first column

# available columns (alphabetical order)
# --------------------------------------
# assetname
# description
# WLmin
# WLmax
# WLmaxmin

myNodeColumns = [
    'assetname',
    'WLmin',
    'WLmax']

myNodeHeaderDE = [
    'AssetName',
    'Min.WSP',
    'Max.WSP']

# export NODE result table to csv and round to 3 decimal places
df_res1dmsmNode.to_csv(rootName + '_Nodes.csv', index_label='MUID', columns = myNodeColumns, header = myNodeHeaderDE, float_format="%.3f")

print(rootName + '_Nodes.csv exported')

Tutor1base.Copy_Nodes.csv exported


In [53]:
# export LINK result table to csv

# Link_ID is index and always exported as the first column

# available columns (aphabetical order)
# -------------------------------------
# assetname
# description
# Diameter
# From_Node_ID
# Ground_level_from_Node
# Ground_level_to_Node
# Invert_level_from_Node
# Invert_level_to_Node
# Length
# Qfull
# Qmax
# Qmax_Qfull
# Qmax_time
# Qmin
# Qmin_time
# Reach_Type
# Slope
# To_Node_ID
# Vmax
# Vmax_time
# Vmin
# Vmin_time
# WLmax_end
# WLmax_end_time
# WLmax_start
# WLmax_start_time


myLinkColumns = [
    'assetname',
    'From_Node_ID',
    'To_Node_ID',
    'Length',
    'Diameter',
    'Slope',
    'Qfull',
    'WLmax_start',
    'WLmax_end',
    'Qmax',
    'Qmax_Qfull',
    'Qmax_time']
    
myLinkHeaderDE = [
    'AssetName',
    'Von Knoten',
    'Nach Knoten',
    'Länge [m]',
    'Profilhöhe [m]',
    'Sohlgefälle [%]',
    'Qvoll [m3/s]',
    'WSPmax obem [m]',
    'WSPmax unten [m]',
    'Qmax [m3/s]',
    'Qmax/Qvoll [ ]',
    'Qmax Zeit']


# export LINK result table to csv and round to 3 decimal places
df_res1dmsmLink.to_csv(rootName + '_Links.csv', index_label='MUID', columns=myLinkColumns, header=myLinkHeaderDE, float_format="%.3f")

print(rootName + '_Links.csv exported')

Tutor1base.Copy_Links.csv exported


In [ ]:
# Wait for input before closing the command window:
input('Press ENTER to close window')